In [1]:
from langchain import LLMChain, PromptTemplate
from langchain_openai import ChatOpenAI

#llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Initialize language model
llm = ChatOpenAI(
        api_key="ollama",
        model="llama3:latest",
        #model="gemma2:latest", #respostas sempre em ingles
        #model="splitpierre/bode-alpaca-pt-br:latest", 
        #model="splitpierre/bode-alpaca-pt-br:13b-Q4_0", 
        base_url="http://localhost:11434/v1",
        temperature=0, 
    )

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided, answer
with "I don't know".
Context: Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.
...
Question: {query}
Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

# Create the LLMChain for the prompt
chain = LLMChain(llm=llm, prompt=prompt_template)

# Set the query you want to ask
input_data = {"query": "What is the main advantage of quantum computing over classical computing?"}

# Run the LLMChain to get the AI-generated answer
response = chain.run(input_data)

print("Question:", input_data["query"])
print("Answer:", response)

/Users/tmedeirosb/Desktop/DEV/LANGCHAIN/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/Users/tmedeirosb/Desktop/DEV/LANGCHAIN/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Question: What is the main advantage of quantum computing over classical computing?
Answer: The main advantage of quantum computing over classical computing is that it can solve complex problems faster.


In [3]:
chain = prompt_template | llm 
response = chain.invoke(input_data)

print("Question:", input_data["query"])
print("Answer:", response.content)


Question: What is the main advantage of quantum computing over classical computing?
Answer: The main advantage of quantum computing over classical computing is that it can solve complex problems faster.


In [4]:
from langchain import FewShotPromptTemplate

examples = [
    {"animal": "lion", "habitat": "savanna"},
    {"animal": "polar bear", "habitat": "Arctic ice"},
    {"animal": "elephant", "habitat": "African grasslands"}
]

example_template = """
Animal: {animal}
Habitat: {habitat}
"""

example_prompt = PromptTemplate(
    input_variables=["animal", "habitat"],
    template=example_template
)

dynamic_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Identify the habitat of the given animal",
    suffix="Animal: {input}\nHabitat:",
    input_variables=["input"],
    example_separator="\n\n",
)

# Create the LLMChain for the dynamic_prompt
chain = dynamic_prompt | llm 

# Run the LLMChain with input_data
input_data = {"input": "tiger"}

response = chain.invoke(input_data)

print(response.content)

Based on the information provided, I would identify the habitat of the given animal as:

Animal: tiger
Habitat: Asian forests and grasslands


In [5]:
prompt_template.save("save_prompt.json")

In [6]:
from langchain.prompts import load_prompt
loaded_prompt = load_prompt("save_prompt.json")

In [7]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template="You are a helpful assistant that translates english to pt-br."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Olá!")

human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human, example_ai, human_message_prompt])

chain = chat_prompt | llm
res = chain.invoke("I love programming.")

print(res.content)

Eu amo programação também!


In [8]:
from langchain import PromptTemplate, FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

chain = few_shot_prompt_template | llm
res = chain.invoke("What's the secret to happiness?")

print(res.content)

I think I can finish that one!

AI: ...is to find your inner pizza, and let it guide you!
